In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_absolute_percentage_error


In [2]:
df = pd.read_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset.csv')


In [3]:
df.columns

Index(['Unnamed: 0', 'regional_master', 'date_field', 'sales', 'flag',
       'week_no'],
      dtype='object')

In [4]:
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df.columns

Index(['regional_master', 'date_field', 'sales', 'flag', 'week_no'], dtype='object')

In [6]:
df.to_csv('/Users/harshitgupta/Desktop/vs /VS-Data-Den/rm_dataset.csv')

In [10]:
df['flag'] = df['flag'].map({'inactive': 0, 'active': 1})
df = df.set_index('date_field')

KeyError: "None of ['date_field'] are in the columns"

In [9]:
df.dtypes

regional_master    float64
sales              float64
flag                 int64
week_no              int64
dtype: object

In [11]:
max_lags = 12
target_variable = 'sales'

In [23]:
df_input = df[['sales' , 'regional_master' ]]



In [24]:
df_train = df_input.iloc[:int(0.8 * len(df_input))]
df_test = df_input.iloc[int(0.8 * len(df_input)):]

In [25]:
unique_codes = df_train['regional_master'].unique()


In [26]:
forecasts_by_category = {}

In [27]:
for code in unique_codes:
    print("Predicting for PTNR_RM_CODE:", code)
    
    # Filter data for the current 'PTNR_RM_CODE'
    df_train_code = df_train[df_train['regional_master'] == code]
    df_test_code = df_test[df_test['regional_master'] == code]
    
    # Train VAR model for the current category
    best_aic = float('inf')
    best_bic = float('inf')
    best_order = 0
    best_model = None
    best_mape = float('inf')

    for i in range(1, max_lags + 1):
        print('Order =', i)
        model = VAR(df_train_code)
        results = model.fit(i)
        print('AIC:', results.aic)
        print('BIC:', results.bic)

        lag_order = results.k_ar
        forecast = results.forecast(df_train_code.values[-lag_order:], len(df_test_code))

        mape_val = mean_absolute_percentage_error(df_test_code[target_variable].values, forecast[:, df_test_code.columns.get_loc(target_variable)])
        print('MAPE:', mape_val)
        print('-' * 40)

        # Update best values if necessary
        if results.aic < best_aic:
            best_aic = results.aic
        if results.bic < best_bic:
            best_bic = results.bic
        if mape_val < best_mape:
            best_mape = mape_val
            best_order = i
            best_model = results

    print('Best order:', best_order)
    print('Best AIC:', best_aic)
    print('Best BIC:', best_bic)
    print('Overall MAPE:', best_mape)
    
    # Forecast for the best model
    lag_order = best_model.k_ar
    forecast = best_model.forecast(df_train_code.values[-lag_order:], len(df_test_code))
    forecast_df = pd.DataFrame(forecast, index=df_test_code.index, columns=df_test_code.columns)
    
    # Store the forecast for this category
    forecasts_by_category[code] = forecast_df


Predicting for PTNR_RM_CODE: 0.0
Order = 1


/Users/harshitgupta/Desktop/vs /VS-Data-Den/.venv/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)


LinAlgError: 2-th leading minor of the array is not positive definite